# Plotly

Для работы с `plotly` используем другие знакомые вам данные — архив объявлений о продаже квартир в Санкт-Петербурге и Ленинградской области из курса «Исследовательский анализ данных». 

Вас просят построить несколько интерактивных графиков, которые покажут:

- как меняется общая площадь квартир в зависимости от расположения;
- как меняется медианная стоимость квартир по годам и месяцам за исключением 2014 года;
- как меняется количество объявлений в зависимости от числа комнат;
- какие пять населённых пунктов, кроме Санкт-Петербурга, лидируют по количеству объявлений;
- как цена за квадратный метр зависит от общей площади квартиры в Санкт-Петербурге и вне его.

Библиотека `plotly` — самостоятельный инструмент. Поэтому импортировать matplotlib или seaborn не потребуется. Понадобится библиотека `pandas`, а также `plotly` в двух вариантах: `plotly.express` и `graph_objects`.

In [3]:
# импортируем библиотеки и загружаем данные
import pandas as pd
import plotly.express as px
from plotly import graph_objects as go
try:
   df = pd.read_csv('/datasets/real_estate_data.csv',sep='\t')
except:
   df = pd.read_csv('https://code.s3.yandex.net/datasets/real_estate_data.csv', sep='\t')

Подготовим данные: удалим пропуски, добавим столбцы с категориями, значениями месяца и года.

In [12]:
# удаляем пропуски 
df['locality_name'] = df['locality_name'].fillna('неизвестно')
# создаём категории для населённых пунктов
df['locality_name_category'] = 'Другое'
df.loc[df['locality_name'] == 'Санкт-Петербург', 'locality_name_category'] = 'Санкт-Петербург'
# переведём first_day_exposition в формат дата-время
df['first_day_exposition'] = pd.to_datetime(df['first_day_exposition'], format=('%Y-%m-%dT%H:%M:%S'))
# добавляем столбец с месяцем
df['month'] = df['first_day_exposition'].dt.month
# добавляем столбец с годом
df['year'] = df['first_day_exposition'].dt.year
df

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition,locality_name_category,month,year,price_m_2
0,20,13000000.0,108.00,2019-03-07,3,2.70,16.0,51.0,8,NaN,...,16028.0,1.0,482.0,2.0,755.0,NaN,Санкт-Петербург,3,2019,120370
1,7,3350000.0,40.40,2018-12-04,1,NaN,11.0,18.6,1,NaN,...,18603.0,0.0,NaN,0.0,NaN,81.0,Другое,12,2018,82920
2,10,5196000.0,56.00,2015-08-20,2,NaN,5.0,34.3,4,NaN,...,13933.0,1.0,90.0,2.0,574.0,558.0,Санкт-Петербург,8,2015,92785
3,0,64900000.0,159.00,2015-07-24,3,NaN,14.0,NaN,9,NaN,...,6800.0,2.0,84.0,3.0,234.0,424.0,Санкт-Петербург,7,2015,408176
4,2,10000000.0,100.00,2018-06-19,2,3.03,14.0,32.0,13,NaN,...,8098.0,2.0,112.0,1.0,48.0,121.0,Санкт-Петербург,6,2018,100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23694,9,9700000.0,133.81,2017-03-21,3,3.70,5.0,73.3,3,NaN,...,4232.0,1.0,796.0,3.0,381.0,NaN,Санкт-Петербург,3,2017,72490
23695,14,3100000.0,59.00,2018-01-15,3,NaN,5.0,38.0,4,NaN,...,NaN,NaN,NaN,NaN,NaN,45.0,Другое,1,2018,52542
23696,18,2500000.0,56.70,2018-02-11,2,NaN,3.0,29.7,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Другое,2,2018,44091
23697,13,11475000.0,76.75,2017-03-28,2,3.00,17.0,NaN,12,NaN,...,10364.0,2.0,173.0,3.0,196.0,602.0,Санкт-Петербург,3,2017,149511


### Площадь квартир в зависимости от расположения

Отобразим, как различается общая площадь квартир в разных категориях населённых пунктов. Построим две гистограммы распределения: одну для Санкт-Петербурга, вторую для других населённых пунктов.

In [5]:
# строим гистограммы
fig = px.histogram(df, # загружаем данные
                   x='total_area', # указываем столбец с данными для оси X
                   color='locality_name_category', # обозначаем категорию для разделения цветом
                   range_x=[0, 200], # ограничиваем ось X
                   title='Распределение общей площади квартиры в зависимости от расположения', # указываем заголовок
                   nbins=1000, # назначаем число корзин
                   barmode='overlay') # выбираем «полупрозрачный» тип отображения столбцов
fig.update_xaxes(title_text='Значение') # подпись для оси X
fig.update_yaxes(title_text='Частота') # подпись для оси Y
fig.show() # выводим график

С помощью параметра `color` можно выбрать категорию для разделения. Вместо одной гистограммы на экране отобразится два графика разных цветов. Какой категории соответствует каждый цвет, можно узнать из легенды графика, которую `plotly` добавит автоматически.

Графики, которые строит `plotly`, интерактивные, поэтому все параметры и значения можно проверить сразу на графике, а также поменять масштаб.

### Изменение медианной стоимости квартир

Чтобы отобразить динамику медианной стоимости квартир, поступим так:

- Подготовим сводную таблицу, в которой сгруппируем данные по месяцу и году, и вычислим медианную стоимость.
- Для отображения динамики лучше всего подходит линейный график, потому его и построим. По оси X укажем месяц, а по оси Y — медианную стоимость. Года выделим цветом.

Покажем, как можно иначе оформлять заголовок и подписывать оси — с помощью параметра `update_layout`.

In [6]:
# готовим сводную таблицу
df_price_time = df.loc[df['year'] != 2014].groupby(['month','year'], as_index=False)[['last_price']].median().round()
# строим линейный график
fig = px.line(df_price_time, # загружаем данные
              x='month', # указываем столбец с данными для оси X
              y='last_price', # указываем столбец с данными для оси Y
              color='year', # обозначаем категорию для разделения цветом
              markers=True) # отображаем маркеры (точки) на графике
# оформляем график
fig.update_layout(title='Медианная стоимость в зависимости от месяца публикации',
                   xaxis_title='Месяц',
                   yaxis_title='Медианная стоимость')
fig.show() # выводим график

### Количество объявлений в зависимости от числа комнат

В этом случае можно использовать круговую диаграмму — pie chart. Это специфический график: лучше добавить побольше информации, чтобы всё было понятно даже неподготовленному человеку.

In [7]:
# исправляем аномалию в данных
df['rooms'] = df['rooms'].replace(0, 1)
# готовим данные для графика
df_rooms = pd.DataFrame(df['rooms'].value_counts()).reset_index()
# строим диаграмму с сегментами
fig = go.Figure(data=[go.Pie(labels=df_rooms['index'], # указываем значения, которые появятся на метках сегментов
                             values=df_rooms['rooms'], # указываем данные, которые отобразятся на графике
                             pull = [0.1, 0])]) # добавляем аргумент, который выделит сегмент-лидер на графике
fig.update_layout(title='Число квартир по количеству комнат', # указываем заголовок графика
                  width=800, # указываем размеры графика
                  height=600,
                  annotations=[dict(x=1.12, # вручную настраиваем аннотацию легенды
                                    y=1.05,
                                    text='Количество комнат',
                                    showarrow=False)])
fig.show() # выводим график

Мы использовали параметр `update_layout` не только для указания заголовка и размера графика. Также мы настроили аннотацию легенды: указали начальные координаты отображения аннотации, добавили текст и убрали специальную стрелку.

С помощью параметра `pull` выделили на графике сегмент-лидер. Числа, которые передают параметру, обозначают, на сколько нужно «оттянуть» лидирующий сегмент от основного графика.

### Лидеры по количеству объявлений

Теперь отобразим пять лидирующих населённых пунктов по количеству объявлений, исключив Санкт-Петербург. Подойдёт самая универсальная диаграмма — столбчатая. Для большей наглядности повернём её набок.

Сначала подготовим данные:

In [8]:
# отфильтруем данные, сгруппируем по городам и посчитаем объявления
df_loc_count = df.loc[df['locality_name_category'] == 'Другое'].groupby('locality_name')[['locality_name']].count()
# переименуем столбец
df_loc_count.columns = ['total_count']
# отсортируем и оставим пять лидеров
df_loc_count = df_loc_count.reset_index().sort_values(by='total_count', ascending=False).head(5)
display(df_loc_count)

,locality_name,total_count
291,посёлок Мурино,522
326,посёлок Шушары,440
3,Всеволожск,398
32,Пушкин,369
13,Колпино,338


Теперь можно строить график:

In [9]:
# строим столбчатую диаграмму 
fig = px.bar(df_loc_count.sort_values(by='total_count', ascending=True), # загружаем данные и заново их сортируем
             x='total_count', # указываем столбец с данными для оси X
             y='locality_name', # указываем столбец с данными для оси Y
             text='total_count' # добавляем аргумент, который отобразит текст с информацией
                                # о количестве объявлений внутри столбца графика
            )
# оформляем график
fig.update_layout(title='ТОП-5 населённых пунктов по количеству квартир',
                   xaxis_title='Количество объявлений',
                   yaxis_title='Населённый пункт')
fig.show() # выводим график

Обратите внимание: параметру `text` передали количество объявлений `total_count`. Оно отобразилось на графике вместе со столбцами. 

Датафрейм df_loc_count хранит топ-5 населённых пунктов, отсортированных по убыванию количества квартир. Однако `px.bar()` по умолчанию отрисовывает столбцы диаграммы в том же порядке, что и строки в датафрейме, но размещает их снизу вверх. Таким образом, если передать методу датафрейм `df_loc_count`, отсортированный по убыванию, то на графике увидим сортировку по возрастанию:

In [10]:
# строим столбчатую диаграмму 
fig = px.bar(df_loc_count, # загружаем данные и заново их сортируем
             x='total_count', # указываем столбец с данными для оси X
             y='locality_name', # указываем столбец с данными для оси Y
             text='total_count' # добавляем аргумент, который отобразит текст с информацией
                                # о количестве объявлений внутри столбца графика
            )
# оформляем график
fig.update_layout(title='ТОП-5 населённых пунктов по количеству квартир',
                   xaxis_title='Количество объявлений',
                   yaxis_title='Населённый пункт')
fig.show() # выводим график

Чтобы этого избежать, методу `px.bar()` можно передать датафрейм `df_loc_count`, отсортированный по возрастанию (`df_loc_count.sort_values(by='total_count', ascending=True)`). Это один из способов «развернуть» такую сортировку.

### Связь между ценой и площадью квартиры

Последнее задание — покажем, как меняется цена за квадратный метр в зависимости от общей площади квартиры в Санкт-Петербурге и вне его.

Отвечать на такие вопросы помогает график scatter. Посчитаем цену квадратного метра для каждого объявления и построим графики разных цветов в зависимости от расположения квартиры.

In [11]:
# в столбце price_m_2 сохраним отношение last_price к areа и приведём к типу int
df['price_m_2'] = (df['last_price'] / df['total_area']).astype('int')
# строим scatter
fig = px.scatter(df,             # загружаем данные
                 x='total_area', # указываем столбец с данными для оси X
                 y="price_m_2",  # указываем столбец с данными для оси Y
                 color='locality_name_category') # обозначаем категорию для разделения цветом
# оформляем график
fig.update_layout(title='Зависимость цены за квадратный метр от общей площади помещения',
                   xaxis_title='Общая площадь',
                   yaxis_title='Цена за квадратный метр')

fig.show() # выводим график

## Выводы

В библиотеке `plotly` множество инструментов и параметров. Как в случае с `matplotlib` и `seaborn`, их сложно описать в одной теме. Поэтому не забывайте пользоваться документацией: в ней вы сможете найти необходимый компонент или метод.

Последний совет: не злоупотребляйте интерактивностью. Лишние элементы могут сбить с толку неподготовленного пользователя вашей презентации. Ищите золотую середину, чтобы дополнительные элементы помогали отвечать на вопросы, а не мешали этому.